In [120]:
from requests import get
import pandas as pd
from API_Keys import quandl
from datetime import datetime, timedelta

# To add new data sources from Quandl
Add your appropriately named URL in the cell below to the right group.

Add that variable to the URL list at the end of the correct group

Adjust range in cells that both merge them and also filter them to account for your addition

For the renaming of merged columns, add your renaming schemata to the correct area.

# This will fail if any step is messed up, or if the quandl data is organized differently than expected. This is hard to automate, potential issues can be expected.

Also, if we need to keep different columns for new data, this will also break.

In [68]:
# Group 1, keep Change, Settle and Volume
CMEgold_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_GC1.json?api_key={quandl}"
CMEsilver_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_SI1.json?api_key={quandl}"
CMEpalladium_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PA1.json?api_key={quandl}"
CMEplatinum_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PL1.json?api_key={quandl}"
CME_TenYear_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_TY1.json?api_key={quandl}"

#Group 2, keep Change, Settle, Wave, and Volume
ICE_USD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_DX1.json?api_key={quandl}"
ICE_ZAR_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_ZR1.json?api_key={quandl}"
# SGX_EuroD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/SGX_ED1.json?api_key={quandl}"

#Group 3, keep Open Interest, last one on list also decides cut off date.
# Last one needs to be shortest weekly separation
Gold_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/088691_FO_ALL.json?api_key={quandl}"
Silver_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/084691_FO_ALL.json?api_key={quandl}"
Palladium_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/075651_FO_ALL.json?api_key={quandl}"
Platinum_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/076651_FO_ALL.json?api_key={quandl}"

In [264]:
url_list = [CMEgold_futures,CMEsilver_futures,CMEpalladium_futures,CMEplatinum_futures,CME_TenYear_futures,ICE_USD_futures,ICE_ZAR_futures,Gold_OpenInt,Silver_OpenInt,Palladium_OpenInt,Platinum_OpenInt]

In [265]:
info_list = [get(url).json() for url in url_list]

In [449]:
df_list = [pd.DataFrame(info["dataset"]["data"],columns=info["dataset"]["column_names"]) for info in info_list]

In [450]:
# Creates a list that functions well in testing
pruned_list = [None]*11

In [451]:
# Cell to prune Group 1
metal_namer = {0:"Gold",1:"Silver",2:"Palladium",3:"Platinum",4:"Ten_Yr_Futures"}
for i in range(5):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume"]]
    pruned_list[i].columns = ["Date",f"{metal_namer[i]}_Change",f"{metal_namer[i]}_Settle",f"{metal_namer[i]}_Volume"]

In [452]:
# Cell to prune Group 2
money_namer = {5:"USD",6:"ZAR"}
for i in range(5,7):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume","Wave"]]
    pruned_list[i].columns = ["Date",f"{money_namer[i]}_Change",f"{money_namer[i]}_Settle",f"{money_namer[i]}_Volume",f"{money_namer[i]}_Wave"]

In [453]:
for i in range(7,11):
    pruned_list[i] = df_list[i][["Date","Open Interest"]]

In [454]:
# Only works for days that are Tuesday releases on their latest and most recent
# will need to add code to adjust for late releases because of Holidays
cut_off_date = df_list[len(df_list)-1]["Date"][len(df_list[len(df_list)-1].index)-1]
cut_off_date = (datetime.strptime(cut_off_date,"%Y-%m-%d") - timedelta(days=8))
cut_off_date = cut_off_date.strftime("%Y-%m-%d")
recent_off_date = df_list[len(df_list)-1]["Date"][0]
recent_off_date = (datetime.strptime(recent_off_date,"%Y-%m-%d") - timedelta(days=4))
recent_off_date = recent_off_date.strftime("%Y-%m-%d")

In [455]:
for i in range(7):
    pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]>=cut_off_date]
    pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]<=recent_off_date]

In [456]:
tuple_list = [((datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7))),(datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7 + 4)))) for w in range(len(df_list[len(df_list)-1].index))]
cut_bins = pd.IntervalIndex.from_tuples(tuple_list,closed="both")

In [457]:
for bin_candidate in range(7):
    bins = pd.cut(pruned_list[bin_candidate]["Date"],cut_bins)
    pruned_list[bin_candidate] = pruned_list[bin_candidate].groupby(bins).mean()
    pruned_list[bin_candidate] = pruned_list[bin_candidate].assign(Date=list(pruned_list[len(pruned_list)-1]["Date"].iloc[::-1]))
    pruned_list[bin_candidate].index.names = ["Business_Week"]

In [461]:
merged_list = [pruned_list[i].\
    merge(pruned_list[7+i], on = "Date", how = "inner").\
    merge(pruned_list[4], on = "Date", how = "inner").\
    merge(pruned_list[5], on = "Date", how = "inner").\
    merge(pruned_list[6], on = "Date", how = "inner").\
    set_index(pruned_list[i].index) for i in range(4)]

In [462]:
merged_list[0].head()

,Gold_Change,Gold_Settle,Gold_Volume,Date,Open Interest,Ten_Yr_Futures_Change,Ten_Yr_Futures_Settle,Ten_Yr_Futures_Volume,USD_Change,USD_Settle,USD_Volume,USD_Wave,ZAR_Change,ZAR_Settle,ZAR_Volume,ZAR_Wave
Business_Week,,,,,,,,,,,,,,,,
"[2006-06-05, 2006-06-09]",NaN,623.46,474.400000,2006-06-13,382474.0,NaN,105.803125,62852.40,0.34200,85.0620,5883.2,NaN,0.01010,6.7417,29.8,NaN
"[2006-06-12, 2006-06-16]",NaN,575.22,121.600000,2006-06-20,390281.0,NaN,105.478125,14248.40,0.04600,86.0820,4010.2,NaN,0.02590,6.8631,80.4,NaN
"[2006-06-19, 2006-06-23]",NaN,580.04,20.600000,2006-06-27,384321.0,NaN,104.568750,294178.80,0.30075,86.1412,3970.4,NaN,0.10125,7.2100,314.5,NaN
"[2006-06-26, 2006-06-30]",NaN,589.92,19187.400000,2006-07-03,392868.0,NaN,104.350000,729784.20,-0.31000,85.7740,3903.6,NaN,-0.05840,7.2927,63.0,NaN
"[2006-07-03, 2006-07-07]",NaN,633.60,42120.333333,2006-07-11,427008.0,NaN,104.597656,758073.25,-0.05200,84.9140,2144.8,NaN,-0.00610,7.1530,164.6,NaN
